In [ ]:
#@title Overview

#@markdown The main purpose of this code is to allow users to query a large body of text relevant to their project easily. It also includes a functionality to visualize the text data in 2 dimensions, which is useful for drawing connections and relatedness between different text chunks.

In [ ]:
#@title User-Defined Variables
search_query = "" #@param
chapter_mapper_folder = "/content/drive/MyDrive/chapter-mapper" #@param
pdf_folder = "Insert relative folder pathname here containing PDF's you want to search #@param
chunk_size = 128 #@param
chunk_overlap = 0.1 #@param
number_results = 3 #@param
openai_api_key = "Insert your OpenAI API Key here" #@param
save_name_suffix = "[with-embeddings].csv" #@param
COMPLETIONS_MODEL = "text-davinci-003" #@param
EMBEDDING_MODEL = "text-embedding-ada-002"#@param

#@markdown ---
#@markdown ### **Help**
#@markdown - **`search_query`** - *`str; The question or information you would like to search for that is related to the text contained in pdf_folder.`*
#@markdown - **`chapter_mapper_folder`** - *`str; The name of the chapter-mapper root folder.`*
#@markdown - **`pdf_folder`** - *`str; The name of the folder which contains all the PDF's you want to analyze.`*
#@markdown - **`chunk_size`** - *`int; How many words you want each chunk of text to contain (approximately). Default is 256.`*
#@markdown - **`chunk_overlap`** - *`float; How much overlap you want each chunk of text to have with the next chunk. Default is 0.1.`*
#@markdown - **`number_results`** - *`int; How many search results you want to see in the plot. Default is 10.`*
#@markdown - **`openai_api_key`** - *`str; Your (free trial) OpenAI API Key`*
#@markdown - **`save_name_suffix`** - *`str; string to add to end of csv file for saving purposes. Default is '[with-embeddings].csv'`*
#@markdown - **`COMPLETIONS_MODEL`** - *`str; The name of the model to use for answering prompts. Default is 'text-davinci-003' from OpenAI.`*
#@markdown - **`EMBEDDING_MODEL`** - *`str; The name of the model to use for obtaining text embeddings. Default is 'text-embedding-ada-002' from OpenAI.`*

In [ ]:
#@title Mount Drive
from google.colab import drive, files
from IPython.display import HTML
import os
import sys
import shutil
import subprocess
output = subprocess.run(["pip", "list"], capture_output=True)
default_packages = output.stdout.decode().strip().split("\n")
drive.mount('/content/drive')
!mkdir /content/drive/MyDrive/python-packages
%cd '/content/drive/MyDrive/python-packages'
sys.path.append('/content/drive/MyDrive/python-packages')

In [ ]:
#@title Install Required Packages
try:
  import openai
except:
  subprocess.run(["pip","install","openai"])
  output = subprocess.run(["pip", "list"], capture_output=True)
  new_packages = output.stdout.decode().strip().split("\n")
  subprocess.run(['pip', 'freeze'], stdout=open(f"{chapter_mapper_folder}/requirements.txt", 'w'))
  non_common_elements_list2 = list(set(new_packages) - set(default_packages))
  non_common_elements = non_common_elements_list2
  for element in non_common_elements:
    element = element.split()[0]+"=="+element.split()[1]
    subprocess.run(["pip", "install",element,"--no-deps",f"--target=/content/drive/MyDrive/python-packages"])
  import openai
try:
  import fitz
except:
  subprocess.run(["pip","install","pymupdf"])
  output = subprocess.run(["pip", "list"], capture_output=True)
  new_packages = output.stdout.decode().strip().split("\n")
  subprocess.run(['pip', 'freeze'], stdout=open(f"{chapter_mapper_folder}/requirements.txt", 'w'))
  non_common_elements_list2 = list(set(new_packages) - set(default_packages))
  non_common_elements = non_common_elements_list2
  for element in non_common_elements:
    element = element.split()[0]+"=="+element.split()[1]
    subprocess.run(["pip", "install",element,"--no-deps",f"--target=/content/drive/MyDrive/python-packages"])
  import fitz
try:
  import tiktoken
except:
  subprocess.run(["pip","install","tiktoken"])
  output = subprocess.run(["pip", "list"], capture_output=True)
  new_packages = output.stdout.decode().strip().split("\n")
  subprocess.run(['pip', 'freeze'], stdout=open(f"{chapter_mapper_folder}/requirements.txt", 'w'))
  non_common_elements_list1 = list(set(default_packages) - set(new_packages))
  non_common_elements_list2 = list(set(new_packages) - set(default_packages))
  non_common_elements = non_common_elements_list1 + non_common_elements_list2
  for element in non_common_elements:
    element = element.split()[0]+"=="+element.split()[1]
    subprocess.run(["pip", "install",element,"--no-deps","--target=/content/drive/MyDrive/python-packages"])
  import tiktoken
subprocess.run(['pip', 'freeze'], stdout=open(f"{chapter_mapper_folder}/requirements.txt", 'w'))

In [ ]:
#@title Change Working Directory to `chapter_mapper_folder`
%cd {chapter_mapper_folder}

In [ ]:
#@title Import Dependencies
import numpy as np
import colorsys
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import textwrap
import os
from utils import batch_embed, ChapterExtractor
from matplotlib import colors as mcolors
from sklearn.manifold import TSNE
from openai.embeddings_utils import get_embedding, cosine_similarity
from typing import List, Dict, Tuple
from tqdm import tqdm
import pickle
openai.api_key = openai_api_key

In [ ]:
#@title Collect Data from all PDF's in `pdf_folder`
df_init = pd.DataFrame()
pdfs = np.unique([os.path.join(pdf_folder,x) for x in os.listdir(pdf_folder) if '.pdf' in x]).tolist()
cs = int(chunk_size)
file_prefix = pdf_folder + f'-cs={cs}-co={chunk_overlap:.2f}-raw-data' 
if not os.path.exists(f'{pdf_folder}/{file_prefix}.csv'):
  for pdf in pdfs:
    print(len(df_init))
    df_temp = ChapterExtractor(pdf,chunk_size,chunk_overlap).get_df()
    df_init = pd.concat([df_init,df_temp])
  df_init.to_csv(f'{pdf_folder}/{file_prefix}.csv',encoding='utf-8-sig')
  df_init = pd.read_csv(f'{pdf_folder}/{file_prefix}.csv')
else:
  print(f'Loading raw data from {pdf_folder}/{file_prefix}.csv...')
  df_init = pd.read_csv(f'{pdf_folder}/{file_prefix}.csv')

In [ ]:
#@title Create Embeddings for the Raw Data
if not os.path.exists(f'{pdf_folder}/{file_prefix}-{save_name_suffix}.csv'):
  batch_embed(df_init,200,'chunk_text') # adds embedding column to the textbook data file
  df_init.to_csv(f'{pdf_folder}/{file_prefix}-{save_name_suffix}.csv',encoding='utf-8-sig')
else:
  df_init = pd.read_csv(f'{pdf_folder}/{file_prefix}-{save_name_suffix}.csv')

In [ ]:
#@title Obtain TSNE Matrix Data for 2D Map Visualization
df = df_init
def feature_matrix(df):
    print("Extracting Embedding Feature Matrix...")
    matrix = df.embeddings.to_list()
    matrix_empty = np.zeros((len(matrix), len(matrix[0])))
    for i in range(len(matrix)):
        try:
            matrix_empty[i, :] = np.array(matrix[i])
        except Exception as e:
            print(i, e)
            print(matrix[i])
            exit()
    matrix = matrix_empty
    return matrix

def toarray(x):
    if isinstance(x, str):
        x = [float(v.strip()) for v in x.strip('[').strip(']').split(',')]
    return x
    
def search_text(df, search_query, n):
    embedding = get_embedding(
        search_query,
        engine=EMBEDDING_MODEL
    )
    df["similarity"] = df.embeddings.apply(lambda x: cosine_similarity(np.asarray(toarray(x), dtype='float64'), np.asarray(embedding, dtype='float64')))
    results = list(df.sort_values("similarity", ascending=False).head(n).description)
    return results

css4_colors = mcolors.CSS4_COLORS

def get_hsv(color_name):
    hexrgb = css4_colors[color_name]
    hexrgb = hexrgb.lstrip("#")   # in case you have Web color specs
    r, g, b = (int(hexrgb[i:i+2], 16) / 255.0 for i in range(0,5,2))
    return colorsys.rgb_to_hsv(r, g, b)

def filter_search(df, search_results):
    df['keep'] = [x in search_results for x in list(df.description)]
    df = df[df['keep'] == True].filter(list(df.columns)[:-1])
    print(df.shape)
    return df

all_titles = list(df.title.unique())
colors = list(css4_colors.keys())
colors = np.random.choice(colors, len(all_titles), False)
colors = sorted(colors, key=get_hsv)
df['color'] = [css4_colors[colors[all_titles.index(i)]] for i in list(df.title)]
dm = {all_titles[i]: colors[i] for i in range(len(all_titles))}

if not os.path.exists(f'{pdf_folder}/{file_prefix}-{save_name_suffix}-[with-TSNE].csv'):
  df['embedding-new'] = df.embeddings.apply(np.array)
  print("Evaluating TSNE on Dataset...")
  tsne = TSNE(n_components=2, perplexity=15, random_state=42, init='random', learning_rate=200)
  matrix = feature_matrix(df)
  vis_dims = tsne.fit_transform(matrix)
  df['x'] = [x for x,y in vis_dims]
  df['y'] = [y for x,y in vis_dims]
  df['description']  = ["<br>".join(textwrap.wrap(d)) for d in list(df.chunk_text)]
  df.to_csv(f'{pdf_folder}/{file_prefix}-{save_name_suffix}-[with-TSNE].csv')
else:
  df = pd.read_csv(f'{pdf_folder}/{file_prefix}-{save_name_suffix}-[with-TSNE].csv')
  df['description']  = ["<br>".join(textwrap.wrap(d)) for d in list(df.chunk_text)]

In [ ]:
#@title Plot Results and Answer Question based on Text Data
if len(search_query) == 0:
  fig = px.scatter(df, x='x', y='y', color='title', hover_data=['title', 'chapter', 'page', 'description'], color_discrete_map=dm, category_orders={'title': all_titles}, template="plotly_dark", title=f"{pdf_folder} Visualized")
  fig.show()
else:
  print("Searching for relevant text...")
  search_results = list(search_text(df, search_query, number_results))
  # print(search_results)
  df_filter = filter_search(df, search_results)
  fig = px.scatter(df_filter, x='x', y='y', color='title', hover_data=['title', 'similarity', 'chapter', 'page', 'description'], color_discrete_map=dm, category_orders={'title': all_titles}, template="plotly_dark", title=f"Text in {pdf_folder} related to '{search_query}'")
  fig.update_layout(hoverlabel=dict(font=dict(family='Arial', size=12, color='black'),align='left'))
  fig.show()

MAX_SECTION_LEN = 1500
SEPARATOR = "\n* "
ENCODING = "cl100k_base"  # encoding for text-embedding-ada-002

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

def vector_similarity(x: List[float], y: List[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, df:pd.DataFrame) -> List[Tuple[float, Tuple[str, str]]]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query,engine=EMBEDDING_MODEL)
    if isinstance(df.embeddings[0],str):
      contexts = df.embeddings.apply(eval)
    else:
      contexts = df.embeddings
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

def construct_prompt(question: str, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, df)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.chunk_text.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know, but here is what I found:"\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.1 because it gives the most predictable, factual answer.
    "temperature": 0.1,
    "max_tokens": 1000,
    "model": COMPLETIONS_MODEL,
}

def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

if len(search_query) > 0:
  print(answer_query_with_context(search_query, df))

In [ ]:
#@title Save the Plot as an HTML File
file_out = f"{pdf_folder}-{search_query}"
go.Figure.write_html(fig,f"{file_out}.html") # write as html or image
files.download(f"{file_out}.html") # download your file and give me a vote my answer

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>